In [49]:
#!pip install plotly --upgrade  # make sure your plotly is up to date

import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [50]:
spectra = loadmat('/content/drive/MyDrive/Spectra_3s_25c.mat')
#spectra = loadmat('Spectra_1s_5c.mat')
spectra.keys()

dict_keys(['__header__', '__version__', '__globals__', 'AmpSAmp', 'Dm_total_save', 'GammaAmp', 'GammaSAmp', 'GammaS_total_save', 'Gamma_total_save', 'M', 'NmaxS', 'dG', 'dGS', 'frequencies', 'n_int', 'noiseAmpScaled', 'omegaS_total_save', 'omega_total_save'])

In [51]:
pd.options.plotting.backend = "plotly" # set default pandas plot to plotly

##Peak Detection on the Channels

In [52]:
all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[0]))
modes = [x[0]*np.shape(spectra['Dm_total_save'])[0] for x in spectra['omega_total_save']]

In [53]:
all_channels.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.157020,0.022974,0.060893,0.090633,0.010773,0.025309,0.068374,0.022459,0.013889,0.019327,0.006349,0.062391,0.030318,0.054057,0.041471,0.057241,0.019839,0.303224,0.072980,0.017175,0.098895,0.125467,0.008303,0.001976,0.068524
1,0.144592,0.023000,0.063589,0.112819,0.008195,0.024602,0.063589,0.016045,0.003098,0.019610,0.006348,0.062192,0.030073,0.056231,0.049025,0.049071,0.027127,0.297582,0.081811,0.010353,0.091706,0.126530,0.007639,0.004120,0.066628
2,0.147236,0.022970,0.061008,0.107335,0.021604,0.027296,0.063752,0.027163,0.004901,0.019518,0.006344,0.062344,0.030359,0.039916,0.034277,0.051305,0.019380,0.298351,0.079102,0.010783,0.098521,0.113965,0.006499,0.008722,0.061215
3,0.149816,0.023015,0.063253,0.110709,0.041911,0.025852,0.063690,0.016569,0.004348,0.019537,0.006347,0.062219,0.030232,0.061577,0.049486,0.046512,0.025455,0.304534,0.071653,0.014433,0.083364,0.110777,0.009341,0.001978,0.073772
4,0.153527,0.022958,0.058932,0.131147,0.001481,0.024155,0.061219,0.015906,0.003410,0.019264,0.006348,0.062198,0.030160,0.039624,0.044207,0.052301,0.029267,0.299466,0.070535,0.013719,0.088285,0.118638,0.005729,0.001044,0.072603


In [68]:
amp=[]
length = np.shape(spectra['Dm_total_save'])[0]
seeker = 0
modes = np.sort(modes)
for j in range(length):
  all_amp = [0]
  while seeker < len(modes) and modes[seeker] <= j+100:
      # print(np.exp(np.square((j-modes[seeker])/100)))
    all_amp.append(1/(np.exp(np.square((j-modes[seeker])/5))))
    seeker += 1
  while seeker < len(modes) and seeker >= 0 and modes[seeker] >= j-100:
    # print(modes[seeker])
    seeker -= 1
  seeker = max(seeker,0)
  amp.append(max(all_amp))
  if j %100000 ==0: print(j)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning:

overflow encountered in exp



100000
200000
300000
400000
500000
600000
700000
800000
900000


##Plotting the Results

In [23]:
def show_spectra(view_range):
  # take subset of plot
  t_modes = np.sort(modes)
  temp_modes = pd.Series(t_modes)[(pd.Series(t_modes) < view_range[1])]
  temp_modes = pd.Series(temp_modes)[(pd.Series(temp_modes) > view_range[0])]
  # temp_peaks = pd.Series(peaks_detected)[(pd.Series(peaks_detected) < view_range[1])]
  # temp_peaks = pd.Series(temp_peaks)[(pd.Series(temp_peaks) > view_range[0])]
  # plot the predicted peaks, modes, and density
  fig = pd.Series(amp).iloc[range(view_range[0],view_range[1])].plot(template='plotly_dark', kind='line')
  for mode in temp_modes:
    fig.add_vline(x=mode, line_width=1, line_color="green") #requires plotly 4.12 and above
  # for peak in temp_peaks:
  #   fig.add_vline(x=peak, line_width=1, line_dash="dash", line_color="red") #requires plotly 4.12 and above
  fig.show()

In [69]:
show_spectra((900000, 901000))

In [73]:
pd.Series(amp).to_csv('labeled_modes')